In [ ]:
import pandas as pd
import random

def generate_apple_product_data(num_records=100):
    """
    Generates synthetic data for an apple product table.

    Args:
        num_records (int): The number of records to generate.

    Returns:
        pandas.DataFrame: A DataFrame containing the synthetic data.
    """

    apple_varieties = [
        "Royal Gala",
        "Fuji",
        "Granny Smith",
        "Golden Delicious",
        "Pink Lady"]

    shelf_lives = [5, 10, 15]
    grades = ["Small", "Medium", "Large"]
    #units_of_measure = ["lbs", "each", "kg", "box", "pallet", "container"] # Added box, pallet and container
    units_of_measure = ["metrictons"] # Added box, pallet and container

    data = []
    for i in range(num_records):
        sku_id = f"APP{i:04d}"  # APP0000, APP0001, ...
        name = f"{random.choice(apple_varieties)} Apple"
        shelf_life = random.choice(shelf_lives)
        grade = random.choice(grades)
        unit_of_measure = random.choice(units_of_measure)

        data.append({
            "SKUID": sku_id,
            "Name": name,
            "ShelfLife": shelf_life,
            "Grade": grade,
            "UnitOfMeasure": unit_of_measure,
        })

    return pd.DataFrame(data)

# Generate and display the synthetic data
apple_data = generate_apple_product_data(45) #generate 20 rows.
print(apple_data)

#Optionally, save to a CSV file.
apple_data.to_csv("products.csv", index=False)

      SKUID                    Name  ShelfLife   Grade UnitOfMeasure
0   APP0000      Granny Smith Apple         10  Medium    metrictons
1   APP0001  Golden Delicious Apple          5   Small    metrictons
2   APP0002        Royal Gala Apple          5   Large    metrictons
3   APP0003              Fuji Apple         15   Large    metrictons
4   APP0004        Royal Gala Apple          5   Large    metrictons
5   APP0005  Golden Delicious Apple         15   Small    metrictons
6   APP0006         Pink Lady Apple          5   Small    metrictons
7   APP0007              Fuji Apple          5  Medium    metrictons
8   APP0008        Royal Gala Apple         15  Medium    metrictons
9   APP0009              Fuji Apple         15   Large    metrictons
10  APP0010      Granny Smith Apple         10  Medium    metrictons
11  APP0011      Granny Smith Apple         10  Medium    metrictons
12  APP0012      Granny Smith Apple          5   Small    metrictons
13  APP0013        Royal Gala Appl

In [ ]:
import pandas as pd
from io import StringIO
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os # Import os module for file path handling

# --- 1. Load & Prepare Data ---

# Harvest Data
harvest_csv = """SupplierID,Country,Apple Variety,Harvest Month,Harvest Quantity
S1,India,Royal Gala,August,600
S1,India,Royal Gala,September,600
S1,India,Fuji,August,200
S1,India,Fuji,September,200
S1,India,Fuji,October,200
S1,India,Golden Delicious,May,800
S1,India,Granny Smith,September,450
S1,India,Granny Smith,October,450
S1,India,Pink Lady,September,350
S1,India,Pink Lady,October,350
S2,South Africa,Royal Gala,January,864
S2,South Africa,Royal Gala,February,864
S2,South Africa,Royal Gala,March,864
S2,South Africa,Royal Gala,April,864
S2,South Africa,Royal Gala,May,864
S2,South Africa,Fuji,January,432
S2,South Africa,Fuji,February,432
S2,South Africa,Fuji,March,432
S2,South Africa,Fuji,April,432
S2,South Africa,Fuji,May,432
S2,South Africa,Golden Delicious,January,576
S2,South Africa,Golden Delicious,February,576
S2,South Africa,Golden Delicious,March,576
S2,South Africa,Golden Delicious,April,576
S2,South Africa,Golden Delicious,May,576
S2,South Africa,Granny Smith,January,648
S2,South Africa,Granny Smith,February,648
S2,South Africa,Granny Smith,March,648
S2,South Africa,Granny Smith,April,648
S2,South Africa,Granny Smith,May,648
S2,South Africa,Pink Lady,January,504
S2,South Africa,Pink Lady,February,504
S2,South Africa,Pink Lady,March,504
S2,South Africa,Pink Lady,April,504
S2,South Africa,Pink Lady,May,504
S3,Chile,Royal Gala,February,2640
S3,Chile,Royal Gala,March,2640
S3,Chile,Fuji,March,1320
S3,Chile,Fuji,April,1320
S3,Chile,Granny Smith,March,1980
S3,Chile,Granny Smith,April,1980
S3,Chile,Golden Delicious,February,1760
S3,Chile,Golden Delicious,March,1760
S3,Chile,Pink Lady,April,1540
S3,Chile,Pink Lady,May,1540
S4,New Zealand,Royal Gala,February,8400
S4,New Zealand,Fuji,March,2100
S4,New Zealand,Fuji,April,2100
S4,New Zealand,Granny Smith,April,3150
S4,New Zealand,Granny Smith,May,3150
S4,New Zealand,Golden Delicious,February,2800
S4,New Zealand,Golden Delicious,March,2800
S4,New Zealand,Pink Lady,April,2450
S4,New Zealand,Pink Lady,May,2450
"""

# Demand Data
demand_csv = """city,customer_id,month,royal_gala,fuji,granny_smith,golden_delicious,pink_lady,total
Berlin,EDEKA,January,78,87,67,81,50,364
Berlin,EDEKA,February,73,81,64,76,49,343
Berlin,EDEKA,March,70,76,62,70,52,329
Berlin,EDEKA,April,67,70,56,64,53,311
Berlin,EDEKA,May,64,64,50,59,56,294
Berlin,EDEKA,June,59,56,48,53,50,266
Berlin,EDEKA,July,56,53,45,48,45,246
Berlin,EDEKA,August,76,62,48,53,42,280
Berlin,EDEKA,September,98,73,53,70,49,343
Berlin,EDEKA,October,106,90,64,81,57,399
Berlin,EDEKA,November,95,98,70,84,59,406
Berlin,EDEKA,December,90,92,73,87,57,399
Berlin,LIDL,January,70,78,60,73,45,325
Berlin,LIDL,February,65,73,58,68,44,306
Berlin,LIDL,March,63,68,55,63,46,294
Berlin,LIDL,April,60,63,50,58,48,278
Berlin,LIDL,May,58,58,45,53,50,263
Berlin,LIDL,June,53,50,43,48,45,238
Berlin,LIDL,July,50,48,40,43,40,220
Berlin,LIDL,August,68,55,43,48,38,250
Berlin,LIDL,September,88,65,48,63,44,306
Berlin,LIDL,October,95,80,58,73,51,356
Berlin,LIDL,November,85,88,63,75,53,363
Berlin,LIDL,December,80,83,65,78,51,356
Berlin,REWE,January,50,56,43,52,32,234
Berlin,REWE,February,47,52,41,49,32,221
Berlin,REWE,March,45,49,40,45,33,212
Berlin,REWE,April,43,45,36,41,34,200
Berlin,REWE,May,41,41,32,38,36,189
Berlin,REWE,June,38,36,31,34,32,171
Berlin,REWE,July,36,34,29,31,29,158
Berlin,REWE,August,49,40,31,34,27,180
Berlin,REWE,September,63,47,34,45,32,221
Berlin,REWE,October,68,58,41,52,37,257
Berlin,REWE,November,61,63,45,54,38,261
Berlin,REWE,December,58,59,47,56,37,257
Hamburg,EDEKA,January,42,48,36,45,27,198
Hamburg,EDEKA,February,39,45,35,42,26,186
Hamburg,EDEKA,March,38,42,33,39,29,180
Hamburg,EDEKA,April,36,39,30,36,29,170
Hamburg,EDEKA,May,35,36,27,33,30,161
Hamburg,EDEKA,June,32,30,26,30,27,144
Hamburg,EDEKA,July,30,29,24,27,24,134
Hamburg,EDEKA,August,42,33,26,30,23,153
Hamburg,EDEKA,September,54,39,29,39,26,186
Hamburg,EDEKA,October,57,48,35,45,32,216
Hamburg,EDEKA,November,51,54,38,47,32,221
Hamburg,EDEKA,December,48,51,39,48,30,216
Hamburg,LIDL,January,34,38,29,36,22,158
Hamburg,LIDL,February,31,36,28,34,20,149
Hamburg,LIDL,March,30,34,26,31,23,144
Hamburg,LIDL,April,29,31,24,29,23,136
Hamburg,LIDL,May,28,29,22,26,24,128
Hamburg,LIDL,June,25,24,20,24,22,115
Hamburg,LIDL,July,24,23,19,22,19,107
Hamburg,LIDL,August,34,26,20,24,18,122
Hamburg,LIDL,September,43,31,23,31,20,149
Hamburg,LIDL,October,46,38,28,36,25,173
Hamburg,LIDL,November,41,43,30,37,25,176
Hamburg,LIDL,December,38,41,31,38,24,173
Hamburg,REWE,January,24,27,20,26,15,112
Hamburg,REWE,February,22,26,20,24,14,105
Hamburg,REWE,March,21,24,19,22,16,102
Hamburg,REWE,April,20,22,17,20,16,96
Hamburg,REWE,May,20,20,15,19,17,91
Hamburg,REWE,June,18,17,14,17,15,82
Hamburg,REWE,July,17,16,14,15,14,76
Hamburg,REWE,August,24,19,14,17,13,87
Hamburg,REWE,September,31,22,16,22,14,105
Hamburg,REWE,October,32,27,20,26,18,122
Hamburg,REWE,November,29,31,21,26,18,125
Hamburg,REWE,December,27,29,22,27,17,122
Munich,EDEKA,January,34,40,29,37,23,164
Munich,EDEKA,February,33,37,28,34,22,153
Munich,EDEKA,March,31,34,26,33,23,147
Munich,EDEKA,April,29,31,25,29,25,140
Munich,EDEKA,May,28,29,22,26,26,132
Munich,EDEKA,June,26,25,20,25,23,119
Munich,EDEKA,July,25,23,19,22,22,110
Munich,EDEKA,August,34,28,20,25,20,127
Munich,EDEKA,September,43,33,23,33,22,153
Munich,EDEKA,October,48,40,28,37,26,180
Munich,EDEKA,November,43,45,31,39,28,186
Munich,EDEKA,December,40,42,33,40,26,181
Munich,LIDL,January,25,30,22,28,17,122
Munich,LIDL,February,24,28,21,25,16,114
Munich,LIDL,March,23,25,20,24,17,109
Munich,LIDL,April,22,23,18,22,18,104
Munich,LIDL,May,21,22,16,20,20,98
Munich,LIDL,June,20,18,15,18,17,89
Munich,LIDL,July,18,17,14,16,16,82
Munich,LIDL,August,25,21,15,18,15,94
Munich,LIDL,September,32,24,17,24,16,114
Munich,LIDL,October,36,30,21,28,20,133
Munich,LIDL,November,32,33,23,29,21,138
Munich,LIDL,December,30,31,24,30,20,135
Munich,REWE,January,22,26,19,24,15,106
Munich,REWE,February,21,24,18,22,14,99
Munich,REWE,March,20,22,17,21,15,95
Munich,REWE,April,19,20,16,19,16,90
Munich,REWE,May,18,19,14,17,17,85
Munich,REWE,June,17,16,13,16,15,77
Munich,REWE,July,16,15,12,14,14,71
Munich,REWE,August,22,18,13,16,13,82
Munich,REWE,September,28,21,15,21,14,99
Munich,REWE,October,31,26,18,24,17,116
Munich,REWE,November,28,29,20,25,18,120
Munich,REWE,December,26,27,21,26,17,117
"""

# Shipping Data
shipping_csv = """Origin Port,Destination Port,Approximate Distance (km),Average Energy Consumption (kWh),Average Cost (EUR),Average Shipping Time (Days)
Jawaharlal Nehru Port Sheva Navi Mumbai,Albert Plesmanweg 240 Rotterdam,21700,325.5,534,30
Port of Cape Town,Albert Plesmanweg 240 Rotterdam,11100,166.5,250,25
Port of San Antonio,Albert Plesmanweg 240 Rotterdam,13900,208.5,702,26
Ports of Auckland,Albert Plesmanweg 240 Rotterdam,17600,264.0,860,58
"""

df_harvest = pd.read_csv(StringIO(harvest_csv))
df_demand = pd.read_csv(StringIO(demand_csv))
df_shipping = pd.read_csv(StringIO(shipping_csv))

# Month mapping
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}
inv_month_map = {v: k for k, v in month_map.items()} # For displaying month names later

# Add numeric month columns
df_harvest['HarvestMonthNum'] = df_harvest['Harvest Month'].map(month_map)
df_demand['MonthNum'] = df_demand['month'].map(month_map)

# Map shipping times to countries (assuming one major port per country for simplicity)
country_port_map = {
    'India': 'Jawaharlal Nehru Port Sheva Navi Mumbai',
    'South Africa': 'Port of Cape Town',
    'Chile': 'Port of San Antonio',
    'New Zealand': 'Ports of Auckland'
}
shipping_dict = df_shipping.set_index('Origin Port')['Average Shipping Time (Days)'].to_dict()

df_harvest['ShippingDays'] = df_harvest['Country'].map(country_port_map).map(shipping_dict)

# --- 2. Aggregate Demand ---
# Melt demand data for easier aggregation
df_demand_melted = df_demand.melt(
    id_vars=['city', 'customer_id', 'month', 'MonthNum'],
    value_vars=['royal_gala', 'fuji', 'granny_smith', 'golden_delicious', 'pink_lady'],
    var_name='Apple Variety',
    value_name='DemandQuantity'
)

# Map apple variety names to match harvest data format
variety_map = {
    'royal_gala': 'Royal Gala',
    'fuji': 'Fuji',
    'granny_smith': 'Granny Smith',
    'golden_delicious': 'Golden Delicious',
    'pink_lady': 'Pink Lady'
}
df_demand_melted['Apple Variety'] = df_demand_melted['Apple Variety'].map(variety_map)

# Calculate total demand per variety per month
monthly_demand = df_demand_melted.groupby(['MonthNum', 'Apple Variety'])['DemandQuantity'].sum().reset_index()
# Convert to dictionary for quick lookup: {(MonthNum, Variety): Quantity}
demand_dict = monthly_demand.set_index(['MonthNum', 'Apple Variety'])['DemandQuantity'].to_dict()

# --- 3. Create Available Supply Pool ---
available_harvest_list = []
for year in [2021]:
    df_year_harvest = df_harvest.copy()
    df_year_harvest['Year'] = year
    df_year_harvest['AvailableQuantity'] = df_year_harvest['Harvest Quantity']
    # Create a unique harvest identifier
    df_year_harvest['HarvestID'] = df_year_harvest.apply(
        lambda row: f"{row['SupplierID']}_{row['Apple Variety']}_{row['HarvestMonthNum']}_{row['Year']}", axis=1
    )
    available_harvest_list.append(df_year_harvest)

available_harvest = pd.concat(available_harvest_list, ignore_index=True)
available_harvest = available_harvest.set_index('HarvestID', drop=False) # Set index for easy lookup and update

# --- 4. Simulate Month by Month ---
purchase_orders = []
po_counter = 1
simulation_years = [2021]
planning_lead_time_months = 3 # Place orders 3 months ahead

print(f"Starting PO Simulation for {simulation_years[0]}-{simulation_years[-1]}...")
print(f"Planning Lead Time: {planning_lead_time_months} months")
print("-" * 30)

for year in simulation_years:
    for sim_month in range(1, 13):
        sim_date = datetime(year, sim_month, 1)
        target_demand_date = sim_date + relativedelta(months=planning_lead_time_months)
        target_month = target_demand_date.month
        target_year = target_demand_date.year

        print(f"--- Simulating Month: {sim_date.strftime('%Y-%m')} ---")
        print(f"Planning for Demand Month: {target_demand_date.strftime('%Y-%m')}")

        # Get demand for the target month
        target_demands = {
            variety: qty for (m, variety), qty
            in demand_dict.items() if m == target_month
        }

        if not target_demands:
            # This handles cases where target month goes beyond Dec (e.g., planning in Nov/Dec 2024 for 2025)
            # Or if demand data is missing for a future month we calculate.
            print(f"No demand data found or required for target month {target_demand_date.strftime('%Y-%m')}. Skipping.")
            continue

        for variety, needed_qty in target_demands.items():
            if needed_qty <= 0: continue

            fulfilled_qty = 0
            print(f"  Target Demand for {variety}: {needed_qty}")

            # Find potential supply: Harvested *before or during* sim_month, correct variety, quantity > 0
            potential_supply = available_harvest[
                (available_harvest['Apple Variety'] == variety) &
                (available_harvest['AvailableQuantity'] > 0) &
                # Harvest must have happened by the simulation date
                ( (available_harvest['Year'] < year) | ((available_harvest['Year'] == year) & (available_harvest['HarvestMonthNum'] <= sim_month)) )
            ].copy() # Copy to avoid SettingWithCopyWarning

            # Sort by harvest date: most recent first (fresher)
            potential_supply = potential_supply.sort_values(by=['Year', 'HarvestMonthNum'], ascending=[False, False])

            if potential_supply.empty:
                 print(f"    WARNING: No available supply found for {variety} harvested by {sim_date.strftime('%Y-%m')} to meet demand for {target_demand_date.strftime('%Y-%m')}")
                 continue

            for harvest_id, supply_row in potential_supply.iterrows():
                if fulfilled_qty >= needed_qty:
                    break # Demand for this variety is met

                order_qty = min(needed_qty - fulfilled_qty, supply_row['AvailableQuantity'])

                if order_qty > 0:
                    # Place the order
                    supplier_id = supply_row['SupplierID']
                    country = supply_row['Country']
                    shipping_days = supply_row['ShippingDays']
                    # Use timedelta for reliable date addition with days
                    expected_arrival_date = sim_date + pd.Timedelta(days=int(shipping_days))

                    po_record = {
                        'PO_ID': f"PO_{po_counter:05d}",
                        'OrderDate': sim_date.strftime('%Y-%m-%d'),
                        'SupplierID': supplier_id,
                        'Country': country,
                        'AppleVariety': variety,
                        'QuantityOrdered': order_qty,
                        'HarvestMonth': inv_month_map[supply_row['HarvestMonthNum']],
                        'HarvestYear': supply_row['Year'],
                        'ExpectedArrivalDate': expected_arrival_date.strftime('%Y-%m-%d'),
                        'DemandMonthTarget': target_demand_date.strftime('%Y-%m'),
                        'SourceHarvestID': harvest_id
                    }
                    purchase_orders.append(po_record)

                    # Update available quantity
                    available_harvest.loc[harvest_id, 'AvailableQuantity'] -= order_qty
                    fulfilled_qty += order_qty
                    po_counter += 1

                    print(f"    Placed PO {po_record['PO_ID']}: {order_qty:.0f} units of {variety} from {supplier_id} ({country}) - Harvested {po_record['HarvestMonth']}/{po_record['HarvestYear']}. Arrival ~{po_record['ExpectedArrivalDate']}")


            if fulfilled_qty < needed_qty:
                print(f"    WARNING: Could not fully meet demand for {variety} for {target_demand_date.strftime('%Y-%m')}. Shortfall: {needed_qty - fulfilled_qty:.0f} units.")

# --- 5. Output Results ---
po_df = pd.DataFrame(purchase_orders)

print("\n" + "=" * 30)
print("Simulation Complete.")
print(f"Total Purchase Orders Generated: {len(po_df)}")
print("=" * 30 + "\n")

# Display the first few and last few POs as an example
print("Sample Purchase Orders Generated:")
if not po_df.empty:
    print(po_df.head().to_string())
    print("...")
    print(po_df.tail().to_string())

    # *** ADDED CODE TO SAVE TO CSV ***
    output_filename = "simulated_purchase_orders_2023_2024.csv"
    try:
        po_df.to_csv(output_filename, index=False)
        print(f"\nPurchase order results successfully saved to: {os.path.abspath(output_filename)}")
    except Exception as e:
        print(f"\nError saving purchase orders to CSV: {e}")
    # ********************************

else:
    print("No purchase orders were generated.")

Starting PO Simulation for 2021-2021...
Planning Lead Time: 3 months
------------------------------
--- Simulating Month: 2021-01 ---
Planning for Demand Month: 2021-04
  Target Demand for Fuji: 344
    Placed PO PO_00001: 344 units of Fuji from S2 (South Africa) - Harvested January/2021. Arrival ~2021-01-26
  Target Demand for Golden Delicious: 318
    Placed PO PO_00002: 318 units of Golden Delicious from S2 (South Africa) - Harvested January/2021. Arrival ~2021-01-26
  Target Demand for Granny Smith: 272
    Placed PO PO_00003: 272 units of Granny Smith from S2 (South Africa) - Harvested January/2021. Arrival ~2021-01-26
  Target Demand for Pink Lady: 262
    Placed PO PO_00004: 262 units of Pink Lady from S2 (South Africa) - Harvested January/2021. Arrival ~2021-01-26
  Target Demand for Royal Gala: 325
    Placed PO PO_00005: 325 units of Royal Gala from S2 (South Africa) - Harvested January/2021. Arrival ~2021-01-26
--- Simulating Month: 2021-02 ---
Planning for Demand Month: 202

In [1]:
!pip install folium

In [2]:
!pip install geopy

Error geocoding Port of Cape Town P O Box 4245 Roggebaai 8000 South Africa: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Port+of+Cape+Town+P+O+Box+4245+Roggebaai+8000+South+Africa&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Could not find coordinates for Jawaharlal Nehru Port Sheva Navi Mumbai 400707 Maharashtra India


Error geocoding Port of San Antonio Barros Luco San Antonio Valparaiso Chile: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Port+of+San+Antonio+Barros+Luco+San+Antonio+Valparaiso+Chile&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Coordinates not found for either Jawaharlal Nehru Port Sheva Navi Mumbai 400707 Maharashtra India or Albert Plesmanweg 240 Rotterdam, 3089KK, The Netherlands. Skipping route.
Coordinates not found for either Port of Cape Town P O Box 4245 Roggebaai 8000 South Africa or Albert Plesmanweg 240 Rotterdam, 3089KK, The Netherlands. Skipping route.
Coordinates not found for either Port of San Antonio Barros Luco San Antonio Valparaiso Chile or Albert Plesmanweg 240 Rotterdam, 3089KK, The Netherlands. Skipping route.
Map saved to shipping_routes.html


In [11]:
import folium
from IPython.display import display

def plot_shipping_routes_with_coords(routes):
    """
    Plots shipping routes on an interactive map using provided coordinates.

    Args:
        routes (dict): A dictionary where keys are origin port coordinates (latitude, longitude)
                       and values are the destination port coordinates (latitude, longitude) for Rotterdam.
    """
    # Create a map centered around Europe with a more suitable tile layer and attribution
    europe_coords = (52.5200, 13.4050)  # Approximate coordinates of Berlin
    m = folium.Map(
        location=europe_coords,
        zoom_start=2,
        tiles="OpenStreetMap",
        attr='Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL.'
    )

    # Add markers and plot routes
    for origin_coords, dest_coords in routes.items():
        # Add origin marker
        folium.Marker(origin_coords, popup=f"Origin: {origin_coords}").add_to(m)

        # Add destination marker
        folium.Marker(dest_coords, popup=f"Destination: {dest_coords}").add_to(m)

        # Plot the route line
        folium.PolyLine([origin_coords, dest_coords], color="blue", weight=2.5, opacity=0.7).add_to(m)

    return m

if __name__ == '__main__':
    # Coordinates for the ports (Latitude, Longitude)
    jawaharlal_nehru_port_coords = (18.9397, 72.9153)
    cape_town_port_coords = (-33.9072, 18.4227)
    san_antonio_port_coords = (-33.5983, -71.6133)
    auckland_port_coords = (-36.8485, 174.7633)
    rotterdam_port_coords = (51.9225, 4.4689)

    shipping_routes_coords = {
        jawaharlal_nehru_port_coords: rotterdam_port_coords,
        cape_town_port_coords: rotterdam_port_coords,
        san_antonio_port_coords: rotterdam_port_coords,
        auckland_port_coords: rotterdam_port_coords
    }

    shipping_map_coords = plot_shipping_routes_with_coords(shipping_routes_coords)

    if shipping_map_coords:
      # Save the map as an HTML file
      shipping_map_coords.save("shipping_routes_coords.html")
      print("Map with coordinates saved to shipping_routes_coords.html")

      # To display the map directly in a Jupyter Notebook, use the display function:
      display(shipping_map_coords)

Map with coordinates saved to shipping_routes_coords.html


In [13]:
import folium

from folium import plugins

import webbrowser

# Define key waypoints (latitude, longitude)

waypoints = [

    (-33.918861, 18.423300),  # Cape Town, South Africa

    (-15.387526, 12.479099),  # Near Angola (West Africa)

    (0.0, -20.0),             # Equatorial Atlantic

    (14.599512, -17.439150),  # Near Senegal

    (35.179554, -6.144410),   # Strait of Gibraltar

    (45.0, 0.0),              # Bay of Biscay (near France)

    (51.508530, -0.125740),   # English Channel (near London)

    (51.922500, 4.479170)     # Rotterdam, Netherlands

]

# Create map centered around mid-point

m = folium.Map(location=[10, 0], zoom_start=3,tiles="OpenStreetMap")

# Add markers for waypoints

for i, (lat, lon) in enumerate(waypoints):

    folium.Marker(location=[lat, lon], popup=f"Stop {i+1}: {lat}, {lon}", icon=folium.Icon(color='blue')).add_to(m)

# Add polyline to connect the waypoints

folium.PolyLine(waypoints, color='blue', weight=2.5, opacity=0.8).add_to(m)

# Add full-screen control

plugins.Fullscreen().add_to(m)

# Save map to an HTML file

map_file = "cape_town_to_rotterdam.html"

m.save(map_file)





In [16]:
import folium

def plot_shipping_routes_with_waypoints(routes):
    """
    Plots shipping routes with waypoints on an interactive map.

    Args:
        routes (dict): A dictionary where keys are origin port coordinates (latitude, longitude)
                       and values are lists of coordinates representing the route,
                       including the destination port (Rotterdam) as the last element.
    """
    # Create a map centered around Europe
    europe_coords = (52.5200, 13.4050)  # Approximate coordinates of Berlin
    m = folium.Map(
        location=europe_coords,
        zoom_start=2,
        tiles="OpenStreetMaps",
        attr='Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL.'
    )

    # Add markers and plot routes
    for origin_coords, route_coords in routes.items():
        # Add origin marker
        folium.Marker(origin_coords, popup=f"Origin: {origin_coords}").add_to(m)

        # Add destination marker (last coordinate in the route)
        dest_coords = route_coords[-1]
        folium.Marker(dest_coords, popup=f"Destination: {dest_coords}").add_to(m)

        # Plot the route line through waypoints
        folium.PolyLine([origin_coords] + route_coords, color="blue", weight=2.5, opacity=0.7).add_to(m)

        # Add markers for waypoints (optional)
        for i, waypoint in enumerate(route_coords[:-1]):
            folium.CircleMarker(waypoint, radius=3, color="gray", fill=True, fill_color="gray", popup=f"Waypoint {i+1}").add_to(m)

    return m

if __name__ == '__main__':
    # Coordinates for the ports (Latitude, Longitude)
    jawaharlal_nehru_port_coords = (18.9397, 72.9153)
    cape_town_port_coords = (-33.9072, 18.4227)
    san_antonio_port_coords = (-33.5983, -71.6133)
    auckland_port_coords = (-36.8485, 174.7633)
    rotterdam_port_coords = (51.9225, 4.4689)

    # Coordinates for waypoints
    suez_canal_coords = (30.57, 32.49)
    gibraltar_strait_coords = (36.07, -5.35)
    panama_canal_coords = (9.10, -79.86)
    cape_good_hope_coords = (-34.35, 18.47)

    shipping_routes_with_waypoints = {
        jawaharlal_nehru_port_coords: [suez_canal_coords, rotterdam_port_coords],
        cape_town_port_coords: [gibraltar_strait_coords, rotterdam_port_coords],
        san_antonio_port_coords: [panama_canal_coords, rotterdam_port_coords],
        auckland_port_coords: [cape_good_hope_coords, gibraltar_strait_coords, rotterdam_port_coords]
    }

    shipping_map_waypoints = plot_shipping_routes_with_waypoints(shipping_routes_with_waypoints)

    if shipping_map_waypoints:
        # Save the map as an HTML file
        shipping_map_waypoints.save("shipping_routes_waypoints.html")
        print("Map with waypoints saved to shipping_routes_waypoints.html")

        # To display the map directly in a Jupyter Notebook, uncomment the line below:
        display(shipping_map_coords)

Map with waypoints saved to shipping_routes_waypoints.html


In [18]:
import folium

from folium import plugins

from IPython.display import display

# Define key waypoints for each route (latitude, longitude)

# Define key waypoints for each route (latitude, longitude)
waypoints_san_antonio = [
    (-33.6, -71.6),  # San Antonio, Chile
    (10.0, -79.5),  # Panama Canal
    (35.0, -5.0),   # Strait of Gibraltar
    (51.9225, 4.4792) # Rotterdam, Netherlands
]

waypoints_auckland = [
    (-36.8485, 174.7633),  # Auckland, New Zealand
    (-12.0, 160.0),        # Pacific Ocean waypoint
    (0.0, -20.0),          # Atlantic Ocean waypoint
    (35.0, -5.0),          # Strait of Gibraltar
    (51.9225, 4.4792)      # Rotterdam, Netherlands
]

waypoints_mumbai = [
    (18.9750, 72.8258),    # Mumbai, India
    (20.0, 80.0),          # Arabian Sea, East of India
    (22.0, 50.0),          # Near Oman
    (35.0, -5.0),          # Strait of Gibraltar
    (51.9225, 4.4792)      # Rotterdam, Netherlands
]

waypoints_cape_town = [
    (-33.918861, 18.423300),  # Cape Town, South Africa
    (-15.387526, 12.479099),  # Near Angola (West Africa)
    (0.0, -20.0),             # Equatorial Atlantic
    (14.599512, -17.439150),  # Near Senegal
    (35.179554, -6.144410),   # Strait of Gibraltar
    (45.0, 0.0),              # Bay of Biscay (near France)
    (51.9225, 4.4792)         # Rotterdam, Netherlands
]

# Create map centered around mid-point

m = folium.Map(location=[10, 0], zoom_start=3, tiles='CartoDB positron')

# Add markers for San Antonio route waypoints

for i, (lat, lon) in enumerate(waypoints_san_antonio):

    folium.Marker(location=[lat, lon], popup=f"San Antonio Stop {i+1}: {lat}, {lon}", icon=folium.Icon(color='red')).add_to(m)

# Add polyline for San Antonio route

folium.PolyLine(waypoints_san_antonio, color='red', weight=2.5, opacity=0.8).add_to(m)

# Add markers for Auckland route waypoints

for i, (lat, lon) in enumerate(waypoints_auckland):

    folium.Marker(location=[lat, lon], popup=f"Auckland Stop {i+1}: {lat}, {lon}", icon=folium.Icon(color='blue')).add_to(m)

# Add polyline for Auckland route

folium.PolyLine(waypoints_auckland, color='blue', weight=2.5, opacity=0.8).add_to(m)

# Add markers for Mumbai route waypoints

for i, (lat, lon) in enumerate(waypoints_mumbai):

    folium.Marker(location=[lat, lon], popup=f"Mumbai Stop {i+1}: {lat}, {lon}", icon=folium.Icon(color='green')).add_to(m)

# Add polyline for Mumbai route

folium.PolyLine(waypoints_mumbai, color='green', weight=2.5, opacity=0.8).add_to(m)

# Add markers for Cape Town route waypoints

for i, (lat, lon) in enumerate(waypoints_cape_town):

    folium.Marker(location=[lat, lon], popup=f"Cape Town Stop {i+1}: {lat}, {lon}", icon=folium.Icon(color='purple')).add_to(m)

# Add polyline for Cape Town route

folium.PolyLine(waypoints_cape_town, color='purple', weight=2.5, opacity=0.8).add_to(m)

# Add full-screen control

plugins.Fullscreen().add_to(m)

# Display map inline in Jupyter Notebook

display(m)

